<table align="left">
  <td>
    <a ref="C:/Users/josep/Snowman/Reforçat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" ref="C:/Users/josep/Snowman/Reforçat.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>



In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os
import random
import time
from IPython.display import clear_output


# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

n=8
m=16

C:\Users\josep\anaconda3\envs\python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\josep\anaconda3\envs\python39\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\josep\anaconda3\envs\python39\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Intern code:

- 0 : out of grid and wall (x and #)
- 1 : small ball
- 2 : medium ball
- 3 : small ball on top of a medium ball
- 4 : large ball
- 5 : small ball on top of a large ball
- 6 : medium ball on top of a large ball
- 7 : small ball on top of a medium ball on top of a large ball
- 8 : grass (,)
- 9 : snow (.)
- 10: character with snow on the floor (p)
- 11: character with grass on the floor (q)


Reconpenses
- 0 moure's sense apretar 
- 0 moure's apretant bola petita
- 0 moure's apretant bola mitjana
- 0 moure's apretant bola grossa
- 100 col.locar bola mitjana sobre bola grossa
- 500 col.locar bola petita sobre boles mitjnes i grosses
- -1 Passar un instant 

Accions prohibides (-100 punts)
- sortir de la quadricula (trepitjar pared)
- fer sortir bola de la quadricula (trepitjar pared la bola)
- fer 2 boles grans
- fer dos boles mitjanes si ja tenim bola gran


Maxim episodi=50 jugades, fins acció prohibida o col.locar tres boles be

In [2]:
error=-10
tonto=-10
cami=0
cim=50
convertir=10
bingo=200

actions=[
    [[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error]],
    [[None,None,None,error],[None,None,None,tonto],['pq',11,3,cim],[None,None,None,tonto],['pq',11,5,cim],[None,None,None,tonto],['pq',11,7,bingo],[None,None,None,tonto],['pq',11,1,cami],['pq',11,2,convertir]],
    [[None,None,None,error],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],['pq',11,6,cim],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],['pq',11,2,cami],['pq',11,4,convertir]],
    [[None,None,None,error],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,2,1,-cim],[None,2,2,-cim]],
    [[None,None,None,error],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],['pq',11,4,cami],['pq',11,4,cami]],
    [[None,None,None,error],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,4,1,-cim],[None,4,2,-cim]],
    [[None,None,None,error],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,4,2,-cim],[None,4,4,-cim]],   
    [[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error]],
    [['pq',11,None,cami],['pq',11,None,cami],['pq',11,None,cami],['pq',11,None,cami],['pq',11,None,cami],['pq',11,None,cami],['pq',11,None,cami],['pq',11,None,tonto],['pq',11,None,cami],['pq',11,None,cami]],
    [['pq',10,None,cami],['pq',10,None,cami],['pq',10,None,cami],['pq',10,None,cami],['pq',10,None,cami],['pq',10,None,cami],['pq',10,None,cami],['pq',10,None,tonto],['pq',10,None,cami],['pq',10,None,cami]]
]



In [3]:
ree='x#,.qp1234567'
reem=[0,0,8,9,11,10,1,2,3,4,5,6,7]
def lleguir_tauler(nom='C:/Users/josep/Snowman/Dades/suy2.txt',n=8,m=16):
    f = open(nom, "r")
    tauler=np.zeros((n,m))
    for row,linea in enumerate(f):
        linea=linea.rstrip('\n\r\t')
        for column,car in enumerate(linea):
            res = ree.find(car)
            tauler[row,column]=reem[res]
    f.close()
    return tauler




# Policy Gradients

To train this neural network we will need to define the target probabilities `y`. If an action is good we should increase its probability, and conversely if it is bad we should reduce it. But how do we know whether an action is good or bad? The problem is that most actions have delayed effects, so when you win or lose points in an episode, it is not clear which actions contributed to this result: was it just the last action? Or the last 10? Or just one action 50 steps earlier? This is called the _credit assignment problem_.

The _Policy Gradients_ algorithm tackles this problem by first playing multiple episodes, then making the actions in good episodes slightly more likely, while actions in bad episodes are made slightly less likely. First we play, then we go back and think about what we did.

Let's start by creating a function to play a single step using the model. We will also pretend for now that whatever action it takes is the right one, so we can compute the loss and its gradients (we will just save these gradients for now, and modify them later depending on how good or bad the action turned out to be):

In [4]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)
enc.fit([[0],[1],[2],[3]])

Q=[0,0,0,0]

In [5]:
def step(t,action): # 0 dreta, 1 baix, 2 esquerra, 3 dalt
    snow=True
    res = np.where(t == 10)
    if np.size(res)==0: # and np.size(res[1])==0:
        snow=False
        res = np.where(t == 11)
    a=res[0]
    b=res[1]
    #print("a,b",a,b,t[a][b])
    inc=[[0,1,0,-1],[1,0,-1,0]]
    seg=[a+inc[0][action],b+inc[1][action]]
    seg2=[a+2*inc[0][action],b+2*inc[1][action]]

    Q[action]=Q[action]+1

    mov=actions[int(t[seg[0],seg[1]])][int(t[seg2[0],seg2[1]])]
    reward=mov[3]
    mov=mov[:3]
    for i,aux in enumerate(mov):
        if aux!=None:
            if aux=='pq':
                if snow:
                    f=9
                else:
                    f=8
            else:
                f=int(aux)

            if i==0:
                t[a,b]=f
            elif i==1:
                t[seg[0],seg[1]]=f
            else:
                t[seg2[0],seg2[1]]=f
    if reward==-77:
        if snow:
            t[a,b]=9
        else:
            t[a,b]=8
        a=random.randint(0,n-1)
        b=random.randint(0,m-1)
        while t[a,b]!=8 and t[a,b]!=9:
            a=random.randint(0,n-1)
            b=random.randint(0,m-1)
        if t[a,b]==8:
            t[a,b]=11
        else:
            t[a,b]=10
    if reward==bingo: # or reward<=tonto:
        done=True
    else:
        done=False

    return (t,reward,done)



In [6]:
def arreglar_taula(t):
    t2=t.copy()
    res = np.where(t2 == 11)
    if np.size(res)!=0:
        t2[res[0],res[1]]=10
    t2=t2/10.
    return t2

In [7]:

from random import choices
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)
enc.fit([[0],[1],[2],[3]])


def possible(t,act,antact,antreward):
    pos=True
    if (act==0 and antact==2) or (act==2 and antact==0) or (act==1 and antact==3) or (act==3 and antact==1):
        if antreward<=cami:
            pos=False
    if act==antact and  antreward<=tonto:
        pos=False
    return pos

def play_one_step(t, model, loss_fn,antact=-3,antreward=-1):
    with tf.GradientTape() as tape:
        left_proba = tf.squeeze(model(arreglar_taula(t)[np.newaxis]))
        proba=left_proba
    
        
        r=np.random.uniform()
        if r>0.10:
            action = choices(population=range(4), k=1, weights=proba)
        else:
            action = choices(population=range(4), k=1)
        pop=set(range(4))
        while not possible(t,action[0],antact,antreward) and pop=={}:
            pop=pop.remove(action[0])
            if len(pop)>0:
                action = choices(population=list(sorted(list(pop))), k=1)

        y_target = enc.transform(np.array([action])).squeeze()
        loss = tf.reduce_mean(loss_fn(y_target, 1-left_proba))
    grads = tape.gradient(loss, model.trainable_variables)
    t, reward, done = step(t,action[0])
    return t, action[0],reward, done, grads

#play_one_step(tauler, model, loss_fn,antact=-3,antreward=-1)

If `left_proba` is high, then `action` will most likely be `False` (since a random number uniformally sampled between 0 and 1 will probably not be greater than `left_proba`). And `False` means 0 when you cast it to a number, so `y_target` would be equal to 1 - 0 = 1. In other words, we set the target to 1, meaning we pretend that the probability of going left should have been 100% (so we took the right action).

Now let's create another function that will rely on the `play_one_step()` function to play multiple episodes, returning all the rewards and gradients, for each episode and each step:

In [8]:


def play_multiple_episodes(n_episodes, n_max_steps, model, loss_fn,ttt):
    all_rewards = []
    all_grads = []
    if ttt%3==0:
        nom='C:/Users/josep/Snowman/Dades/suy.txt'
    elif ttt%3==1:
        nom='C:/Users/josep/Snowman/Dades/suy.txt'
    else:
        nom='C:/Users/josep/Snowman/Dades/suy.txt'
    
    tauler=lleguir_tauler(nom,n=n,m=m)
    maxim=-1000000
    for episode in range(n_episodes):
        t=tauler.copy()
        '''snow=True
        res = np.where(t == 10)
        if np.size(res[0])==0 and np.size(res[1])==0:
            snow=False
            res = np.where(t == 11)
        ([a],[b])=res
        print(res)
        if snow:
            t[a,b]=9
        else:
            t[a,b]=8
        a=random.randint(0,n-1)
        b=random.randint(0,m-1)
        while t[a,b]!=8 and t[a,b]!=9:
            a=random.randint(0,n-1)
            b=random.randint(0,m-1)
        if t[a,b]==8:
            t[a,b]=11
        else:
            t[a,b]=10'''

        antact=-3
        reward=-1
        current_rewards = []
        current_grads = []
        for step in range(n_max_steps):
            t, antact,reward, done, grads = play_one_step(t, model, loss_fn,antact,reward)
            current_rewards.append(reward)
            current_grads.append(grads)
            if done:
                break
        if sum(current_rewards)>maxim:
            maxim=sum(current_rewards)
        all_rewards.append(current_rewards)
        all_grads.append(current_grads)
    return all_rewards, all_grads, maxim



The Policy Gradients algorithm uses the model to play the episode several times (e.g., 10 times), then it goes back and looks at all the rewards, discounts them and normalizes them. So let's create couple functions for that: the first will compute discounted rewards; the second will normalize the discounted rewards across many episodes.

In [9]:
def discount_rewards(rewards, discount_rate):
    discounted = np.array(rewards)
    for step in range(len(rewards) - 2, -1, -1):
        discounted[step] += discounted[step + 1] * discount_rate
    return discounted

def discount_and_normalize_rewards(all_rewards, discount_rate):
    all_discounted_rewards = [discount_rewards(rewards, discount_rate)
                              for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean) / reward_std
            for discounted_rewards in all_discounted_rewards]

Say there were 3 actions, and after each action there was a reward: first 10, then 0, then -50. If we use a discount factor of 80%, then the 3rd action will get -50 (full credit for the last reward), but the 2nd action will only get -40 (80% credit for the last reward), and the 1st action will get 80% of -40 (-32) plus full credit for the first reward (+10), which leads to a discounted reward of -22:

To normalize all discounted rewards across all episodes, we compute the mean and standard deviation of all the discounted rewards, and we subtract the mean from each discounted reward, and divide by the standard deviation:

In [10]:
n_iterations = 2500
n_episodes_per_update = 50
n_max_steps = 50
discount_rate = 0.90

In [11]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
loss_fn = keras.losses.categorical_crossentropy

In [12]:
tf.keras.backend.clear_session()
np.random.seed(13)
tf.random.set_seed(13)
model_base = tf.keras.models.load_model('C:/Users/josep/Snowman/encoder.h5')
model_base.trainable=True

model=tf.keras.models.Sequential([
    model_base,
    tf.keras.layers.Dense(64,activation='tanh'),
    tf.keras.layers.Dense(4,activation='softmax')
])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Functional)         (None, 512)               19200     
_________________________________________________________________
dense (Dense)                (None, 64)                32832     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 52,292
Trainable params: 52,100
Non-trainable params: 192
_________________________________________________________________
None


In [13]:
def prova():
    tauler=lleguir_tauler('C:/Users/josep/Snowman/Dades/suy.txt',n=n,m=m)
    print(tauler)
    time.sleep(2)
    clear_output(wait=True)
    sum_rewards=0

    for i in range(50):    
        action=np.argmax(model.predict(arreglar_taula(tauler)[np.newaxis,:,:]).squeeze())
        clear_output(wait=True)
        tauler, reward, done = step(tauler,action)
        sum_rewards+=reward
        print(action,i,sum_rewards)
        print(tauler)
        time.sleep(1)
    #clear_output(wait=True)

In [14]:

#model = tf.keras.models.load_model('snowmangradultim.h5')
 
for iteration in range(n_iterations):
    if (iteration+1)%100==0:
        prova()
    
    all_rewards, all_grads, maxim = play_multiple_episodes(n_episodes_per_update, n_max_steps, model, loss_fn,iteration)

    total_rewards = sum([sum(ar) for ar in all_rewards])                   # Not shown in the book
    print("\rIteration: {}, mean rewards: {:.1f} Mov: {} maxim: {}     ".format(iteration, total_rewards / n_episodes_per_update,Q,maxim), end="") # Not shown

    all_final_rewards = discount_and_normalize_rewards(all_rewards, discount_rate)
    
    all_mean_grads = []
    for var_index in range(len(model.trainable_variables)):
        mean_grads = tf.reduce_mean(
            [final_reward * all_grads[episode_index][step][var_index]
             for episode_index, final_rewards in enumerate(all_final_rewards)
                 for step, final_reward in enumerate(final_rewards)], axis=0)
        all_mean_grads.append(mean_grads)
        

        
    optimizer.apply_gradients(zip(all_mean_grads, model.trainable_variables))




3 49 -500
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  8.  8. 11.  9.  9.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  8.  8.  1.  8.  8.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  8.  9.  9.  9.  8.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  8.  9.  4.  1.  8.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
Iteration: 934, mean rewards: -338.0 Mov: [98787, 284267, 94989, 1854794] maxim: 240      

In [ ]:
Q=[0,0,0,0]
prova() 


In [ ]:
print(tauler)

In [ ]:

model.save('snowmangradultim2.h5')